In [15]:
%pwd

'/home/jovyan/workspace/chicken-disease-classification'

In [2]:
import os
os.chdir('../')

In [3]:
%pwd

'/home/jovyan/workspace/chicken-disease-classification'

## Entities 

In [16]:
from dataclasses import dataclass
from pathlib import Path

In [17]:
@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list

@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

## configurtions manager in src/config

In [18]:
from cnnClassifier.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from cnnClassifier.utils.common import read_yaml, create_directories
import tensorflow as tf

In [19]:
class ConfigurationManager():
    def __init__(
            self,
            config_filepath=CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH
    ):
        print("Configuration Manager Initiated")
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_prepare_callback_config(self) -> PrepareCallbacksConfig:
        
        create_directories([Path(self.config.prepare_callbacks.root_dir),
                            Path(self.config.prepare_callbacks.tensorboard_root_log_dir),
                            Path(self.config.prepare_callbacks.checkpoint_model_filepath)])
        return PrepareCallbacksConfig(
            root_dir = Path(self.config.prepare_callbacks.root_dir),
            tensorboard_root_log_dir = Path(self.config.prepare_callbacks.tensorboard_root_log_dir),
            checkpoint_model_filepath= Path(self.config.prepare_callbacks.checkpoint_model_filepath)
        )
    

    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = self.config.data_ingestion.unzip_dir
        create_directories([Path(training.root_dir)])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=str(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config


## Components

In [20]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf # to call VGG16 from keras
import time

In [21]:
class PrepareCallback:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config
    
    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}"
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    
    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=str(self.config.checkpoint_model_filepath),
            save_best_only=True
        )
    
    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]
    
    

In [22]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config
        self.model = self.get_base_model()

    def get_base_model(self):
        model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )
        model.compile(optimizer='adam',
                        loss='categorical_crossentropy'
                        ,metrics=['accuracy'])
        return model


    def train_valid_generator(self):
        datagenerator_kwargs = dict(
            rescale=1./255,
            validation_split=.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

    @staticmethod
    def save_model(path: Path, model: tf.keras.models.Model):
        model.save(path)

    def train(self, callback_list: list):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator,
            callbacks=callback_list
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )


## Pipeline

In [23]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()

    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train(
        callback_list=callback_list
    )
except Exception as e:
    raise e


Configuration Manager Initiated
[ 2023-09-25 06:44:10,919: INFO: common:  yaml file: config/config.yaml loaded successfully]
[ 2023-09-25 06:44:10,921: INFO: common:  yaml file: params.yaml loaded successfully]
[ 2023-09-25 06:44:10,922: INFO: common:  directory: artifacts created successfully]
[ 2023-09-25 06:44:10,922: INFO: common:  directory: artifacts created successfully]
[ 2023-09-25 06:44:10,923: INFO: common:  directory: artifacts/prepare_callbacks created successfully]
[ 2023-09-25 06:44:10,924: INFO: common:  directory: artifacts/prepare_callbacks created successfully]
[ 2023-09-25 06:44:10,925: INFO: common:  directory: artifacts/prepare_callbacks/tensorboard_log_dir created successfully]
[ 2023-09-25 06:44:10,925: INFO: common:  directory: artifacts/prepare_callbacks/tensorboard_log_dir created successfully]
[ 2023-09-25 06:44:10,926: INFO: common:  directory: artifacts/prepare_callbacks/checkpoint_dir created successfully]
[ 2023-09-25 06:44:10,927: INFO: common:  directo

2023-09-25 06:44:11.011933: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-09-25 06:44:11.018234: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-09-25 06:44:11.018486: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-09-25 06:44:11.019022: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the approp

[ 2023-09-25 06:44:12,150: WARNING: hdf5_format:  No training configuration found in the save file, so the model was *not* compiled. Compile it manually.]
[ 2023-09-25 06:44:12,394: WARNING: hdf5_format:  No training configuration found in the save file, so the model was *not* compiled. Compile it manually.]
Found 160 images belonging to 2 classes.
Found 640 images belonging to 2 classes.


2023-09-25 06:44:14.103666: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8100
2023-09-25 06:44:14.775345: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-09-25 06:44:14.775858: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-09-25 06:44:14.775891: W tensorflow/stream_executor/gpu/asm_compiler.cc:80] Couldn't get ptxas version string: INTERNAL: Couldn't invoke ptxas --version
2023-09-25 06:44:14.776356: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-09-25 06:44:14.776423: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] INTERNAL: Failed to launch ptxas
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.


40/40 [==============================] - ETA: 0s - loss: 0.3189 - accuracy: 0.8687[ 2023-09-25 06:44:26,266: WARNING: save:  Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 13). These functions will not be directly callable after loading.]
[ 2023-09-25 06:44:26,726: INFO: builder_impl:  Assets written to: artifacts/prepare_callbacks/checkpoint_dir/assets]
40/40 [==============================] - 14s 255ms/step - loss: 0.3189 - accuracy: 0.8687 - val_loss: 0.2245 - val_accuracy: 0.9375
